# ![Ray Data DB API 2 Connector](images/dbapi2_connector_logo.png)
This user guide walks through the basics of reading and writing data with the Ray DB API 2 connector.

The Ray DB API2 connector enables parallel read and write to and from SQL compliant datastores with Ray datasets. The connector utilizes the Python DB API 2.0 specification implemented by most databases. DB API 2 providers include:

* SQLite 3 (this example)
* Databricks (see [example](working_with_databricks.ipynb))
* Snowflake (see [example](working_with_snowflake.ipynb))
* Google Big Query
* Microsoft SQL Server
* Teradata
* Postgres
* MySQL
* Oracle
* Many more ...

## Initialize ray
Ray will automatically be initialized with defaults when calling any ray or ray dataset methods. To specify configuration, add the below.

In [1]:
import ray, logging
logging.basicConfig(level=logging.ERROR) # only show errors

if not ray.is_initialized():
    ray.init()

2023-02-26 22:05:56,375	INFO worker.py:1242 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
find: ‘.git’: No such file or directory
2023-02-26 22:05:56,677	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.36.75:9031...
2023-02-26 22:05:56,683	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_vnmb5jgl4z6q98h61dx25rccju/services?redirect_to=dashboard 
2023-02-26 22:05:56,688	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_d1ad1ea6b191d25ffe45b12747a1b1d2.zip' (0.83MiB) to Ray cluster...
2023-02-26 22:05:56,699	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_d1ad1ea6b191d25ffe45b12747a1b1d2.zip'.


## Connection properties
The databsae connection properties need to be provided to the data source upon creation. These properties are documented by the database provider, but for this  example we will use SQLite implementation which is provided in Python.

Below is an example of loading properties from the environment, and filtering them by the 'SQLITE_' prefix.

In [2]:
import os
connect_props = {
    key.replace('SQLITE_','').lower(): value 
    for key,value in os.environ.items() if 'SQLITE_' in key
}

# create the database directory if needed
dirname, filename = os.path.split(connect_props['database'])
if not os.path.exists(dirname):
    os.makedirs(dirname)

print('Connection properties:')
print('\n'.join(connect_props.keys()))

Connection properties:
database


## Reading
Ray will use SQLite Python API to read in parallel into a Ray cluster. The created Ray datasets is composed of PyArrow dataframes that are spread across the Ray cluster to allow for the distributed operations required in machine learning.

![DB API 2 read](images/dbapi2_read.png)


### Read from tables
In order to read an entire table into a a Ray cluster, utilize the Ray data `read_dbapi2` method. The code below will read in a sample table from an SQLite sample database.

In [3]:
from sqlite3 import connect as connect_fn
from ray.data import read_dbapi2

# create sample data
SAMPLE_SIZE = 150000
with connect_fn(**connect_props) as con:
    con.execute('DROP TABLE IF EXISTS source')
    con.execute('CREATE TABLE source(int_val, str_val, flt_val, bool_val)')
    con.commit()
    data = [[int(i),str(i),float(i),i%2==0] for i in range(0,SAMPLE_SIZE)]
    con.executemany('INSERT INTO source VALUES (?, ?, ?, ?)', data)

# read the entire table
ds = read_dbapi2(connect_fn, connect_props, table='source') 

# get all partitions
ds = ds.fully_executed()

# display the first 3 results
print('count:',ds.count())
ds.limit(3).to_pandas()

Read progress: 100%|██████████| 16/16 [00:02<00:00,  6.84it/s]


count: 150000


,int_val,str_val,flt_val,bool_val
0,0,0,0.0,1
1,1,1,1.0,0
2,2,2,2.0,1


### Read with a query
For more control over columns and rows read, as well as joining data from multiple tables, a query can be specified instead of a table name. 

In [4]:
QUERY = 'SELECT int_val, str_val FROM source WHERE int_val >= 3 AND int_val <= 5'

# read the result of the query
ds2 = read_dbapi2(connect_fn, connect_props, query=QUERY)

# display the first 3 results
print('count:',ds2.count())
ds2.limit(3).to_pandas()

2023-02-26 22:06:05,628	WARNING read_api.py:331 -- ⚠️  The number of blocks in this dataset (3) limits its parallelism to 3 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


count: 3


Read progress: 100%|██████████| 3/3 [00:00<00:00, 2594.41it/s]


,int_val,str_val
0,3,3
1,4,4
2,5,5


### Additional read parameters
For reading from Snowflake, underlying Python API arguments are also available. The `args` parameter will be passed to the underlying
execute method. Other drivers may require kwargs whihc can be passed as key word arguments to the `read_dbapi2` method.

The code below uses the args to specify parameterss to be used by SQLite when executing the query.

In [5]:
QUERY = 'SELECT int_val, str_val FROM source WHERE int_val > ?'

ds3 = read_dbapi2(connect_fn, connect_props, query=QUERY, query_args=[[5]])

print('count:',ds3.count())
ds3.limit(3).to_pandas()

count: 149994


Read progress: 100%|██████████| 1/1 [00:00<00:00, 1144.73it/s]


,int_val,str_val
0,6,6
1,7,7
2,8,8


## Writing
The Ray DB API 2 connector will use the DB API driver to write each partition of data in parallel. Each partition of data in the Ray dataset will have a write task that writes in parallel to Snowflake.
![DB API 2 write](images/dbapi2_write.png)

### Write to tables
In order to write a dataset into database table, use the `write_dbapi2` method of the dataset object. Repartition the dataset prior to calling this method in order to set the number of write tasks.

In [6]:
# create the destination table
with connect_fn(**connect_props) as con:
    con.execute('DROP TABLE IF EXISTS destination')
    con.execute('CREATE TABLE destination(int_val, str_val, flt_val, bool_val)')
    con.execute('DROP TABLE IF EXISTS destination2')
    con.execute('CREATE TABLE destination2(int_val, str_val, flt_val, bool_val)')

The example below writes the previously read data into a new database table that are created using the Snowflake Python API.

In [7]:
# write the dataset to the table 
ds.write_dbapi2(connect_fn, connect_props, table='destination')

# display the first 3 results
ds4 = read_dbapi2(connect_fn, connect_props, table='destination')
print('count:',ds4.count())
ds4.limit(3).to_pandas()

2023-02-26 22:06:07,302	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[write]
write: 100%|██████████| 16/16 [00:03<00:00,  4.40it/s]


count: 150000


Read progress: 100%|██████████| 1/1 [00:00<00:00, 886.18it/s]


,int_val,str_val,flt_val,bool_val
0,9375,9375,9375.0,0
1,9376,9376,9376.0,1
2,9377,9377,9377.0,0


### Writing with stage tables
Some databases may lock tables during writing which would significantly increase the overal write time. To get around this, the write mode can be set to stage, wich will cause each partition to be writtent its own stage table. After all data is written to stage tables, they are then copied to the main table. In order for this to work, the database must support the defualt `CREATE TABLE <STAGE> LIKE <DESTINATION>` semantics. 

![DB API 2 write](images/dbapi2_write_staged.png)

> Note: SQLite doesn't support `CREATE TABLE <STAGE> LIKE <DESTINATION>`. Since these semantics are not supported, the below overrides the prepare query to provide the template of sql for creating the stage table. `{table}` will be replaced with the destination table name, and `{block_id}` will be replaced with a unique id for the data block/partition. Many databases may support this, so the `write_queries` parameter would not be needed.

In [8]:
# write the dataset to the table
ds.write_dbapi2(
    connect_fn,
    connect_props, 
    table='destination2',
    mode='stage'
)

# display the first 3 results
ds5 = read_dbapi2(connect_fn, connect_props, table='destination2')
print('count:',ds5.count())
ds5.limit(3).to_pandas()

2023-02-26 22:06:12,228	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[write]
write: 100%|██████████| 16/16 [00:06<00:00,  2.42it/s]


count: 150000


Read progress: 100%|██████████| 1/1 [00:00<00:00, 1197.69it/s]


,int_val,str_val,flt_val,bool_val
0,0,0,0.0,1
1,1,1,1.0,0
2,2,2,2.0,1


## Advanced Usage
If more low level access to the Ray DB API 2 connector is needed, the underlying `DBAPI2Connector` and `DBAPI2Datasource` can be used.

### DB API 2 Connector
The `DBAPI2Connector` class holds the connection properties and logic required to establish a connection with a DB API 2 driver for a database. Internally it calls the native Python dirver API in order to read and write from and to tables in parallel across the cluster. The datasource uses the DB API 2 `execute` and `executemany` methods to enable parallel read and writes of data.

The connector is also a Python context manager, and utilize `with` semantics to define when a connection should be established, db operations commited to the database, and the connection closed. 

The code below will read from a sample table using the connector to manage the connection.

In [9]:
from ray.data.datasource import DBAPI2Connector

# query the number of rows, using the connection context to
# manage transactions
with DBAPI2Connector(connect_fn, **connect_props) as con:
    con.query_int('''
        CREATE TABLE IF NOT EXISTS destination3
        AS SELECT * FROM destination LIMIT 0
    ''')

Alternatively, you can use `try` blocks with the connector's `open`, `commit` and `close` methods. 

In [10]:
connector = DBAPI2Connector(connect_fn, **connect_props)
try:
    connector.open()
    count = connector.query_int(f'SELECT COUNT(*) FROM source')
finally:
    connector.close()
    
print(count)

150000


### DB API 2 Datasource
The DB API 2 datasource can be used with the Ray data `read_datasource` and `write_datasource` methods to read and write to databases using the distibuted processing capabilities of Ray data. The datasource uses the DBAPI2Connector class internally.

Below is an exmaple of creating the datasource using the previously defined connect properties, and then using it to read and write.

In [11]:
from ray.data.datasource import DBAPI2Datasource
from ray.data import read_datasource

# create a datasource from a connector
datasource = DBAPI2Datasource(connector)

# use read_datasource to read
ds = read_datasource(
    datasource, 
    table='source'
)
 
ds.limit(3).to_pandas()

Read progress: 100%|██████████| 1/1 [00:00<00:00, 1174.22it/s]


,int_val,str_val,flt_val,bool_val
0,0,0,0.0,1
1,1,1,1.0,0
2,2,2,2.0,1


In [12]:
# use write_datasource to write
ds.write_datasource(datasource, table='destination3')

ds6 = read_dbapi2(connect_fn, connect_props, table='destination3')
print('count:',ds6.count())
ds6.limit(3).to_pandas()

2023-02-26 22:06:22,456	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->write]
read->write: 100%|██████████| 16/16 [00:06<00:00,  2.46it/s]


count: 150000


Read progress: 100%|██████████| 1/1 [00:00<00:00, 1114.91it/s]


,int_val,str_val,flt_val,bool_val
0,0,0,0.0,1
1,1,1,1.0,0
2,2,2,2.0,1


### DML and DDL
The connector can also be used for any DDL or DML operations you would normally execute through the DB Native Python API. These operations just pass through to the underlying API. 

The code below will create the objects needed for writing to tables. Note that a commit is issued between the queries so the DDL operation executes prior to the next one that is dependent. An alternative is to use two `with` blocks to define transaction boundaries.

In [13]:
with connector as con:
    con.query(f'''
        DROP TABLE IF EXISTS destination;
        DROP TABLE IF EXISTS  destination2;
        DROP TABLE IF EXISTS  destination3;
        DROP TABLE  IF EXISTS source
    ''')

(autoscaler +45s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +45s) Adding 1 node(s) of type worker-node-type-0.
